In [14]:
import os
import pandas as pd

In [15]:
# Define the path
path = os.path.join("GitHub", "DataProcessing")

# Print the path
print(path)


GitHub\DataProcessing


In [16]:
pwd

'C:\\Users\\HP4\\Documents\\GitHub\\DataProcessing'

In [17]:
TestData = pd.read_json("..\DataProcessing\TestData\message.json")